# ROME Capstone Project - The Battle of the Neighborhoods (Week 1/2)

![Rome](img/Rome.jpg)

## Table of contents
* [Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a dentist office. Specifically, this report will be targeted to stakeholders interested in opening an **Dentist Office** in **Rome**, Italy.

Since there are lots of dentists in Rome we will try to detect **locations that are not already crowded with dentists**. We are also particularly interested in **areas with no Italian in vicinity**. 

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing dentists in the neighborhood (any type of dentist)
* Density of dentists in the neighborhood, if any

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Geolocator** (https://github.com/geopy/geopy)
* number of dentists and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Rome center will be obtained using **Geolocator** of well known Rome location (Piazza del Campidoglio)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Rome city center.

Let's first find the latitude & longitude of Rome city center, using specific, well known address and Geolocator geocoding API.

In [1]:
address = 'Rome, Italy'
import geopy
from geopy.geocoders import Nominatim


In [2]:
def get_coordinates(address):
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        return [latitude, longitude]
    except:
        return [None, None]
address = 'Rome, Italy'
rome_center = get_coordinates(address)
print('Coordinate of {}: {}'.format(address, rome_center))

Coordinate of Rome, Italy: [41.8933203, 12.4829321]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Piazza del Campidoglio. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
#!pip install shapely
#import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Rome center longitude={}, latitude={}'.format(rome_center[1], rome_center[0]))
x, y = lonlat_to_xy(rome_center[1], rome_center[0])
print('Rome center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Rome center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Rome center longitude=12.4829321, latitude=41.8933203
Rome center UTM X=291187.2193899718, Y=4640995.599190208
Rome center longitude=12.482932100000001, latitude=41.89332029999999


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
rome_center_x, rome_center_y = lonlat_to_xy(rome_center[1], rome_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = rome_center_x - 6000
x_step = 600
y_min = rome_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
area_id=[]
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(rome_center_x, rome_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
            area_id.append('area_'+str(i)+'_'+str(j))

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [5]:
#!pip install folium
import folium

In [6]:
map_rome = folium.Map(location=rome_center, zoom_start=12,tiles='Stamen Toner')
folium.Marker(rome_center, popup='Piazza del Campidoglio').add_to(map_rome)
for lat, lon ,area in zip(latitudes, longitudes,area_id):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rome) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rome)
    #folium.Marker([lat, lon],popup=area).add_to(map_rome)
map_rome

![Rome](img/1.jpg)

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Piazza del Campidoglio. 

Let's now use Geolocator to get approximate addresses of those locations.

In [7]:
def get_address(latitude, longitude):
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.reverse("{}, {}".format(latitude,longitude))
        return location
    except:
        return None

In [8]:
addr = get_address(rome_center[0], rome_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(rome_center[0], rome_center[1], addr))

Reverse geocoding check
-----------------------
Address of [41.8933203, 12.4829321] is: Piazza del Campidoglio, Via di San Pietro in Carcere, Rione X Campitelli, Municipio Roma I, Roma, Roma Capitale, 00184, Italia


In [9]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = str(address).replace(', Italia', '').replace(', Roma Capitale', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [10]:
addresses[150:170]

['Via Arriano, Pigneto, Quartiere VII Prenestino-Labicano, Roma, 00176',
 'Via Isabella De Rosis, Pigneto, Quartiere VII Prenestino-Labicano, Roma, 00176',
 'Cisco Collatino, Via dei Gordiani, Quartiere VII Prenestino-Labicano, Roma, 00177',
 'Società Sportiva Dilettantistica Vita, 66, Via del Fontanile Arenato, Suburbio VIII Gianicolense, Roma, 00163',
 'Residence Aurelia Antica, Via Aurelia Antica, Suburbio VIII Gianicolense, Roma, 00164',
 'Viale Maria Callas, Quartiere XII Gianicolense, Roma, 00164',
 'Area Cani, Viale Ada Gobetti, Monteverde, Quartiere XII Gianicolense, Roma, 00165',
 'Viale Miriam Mafai, Monteverde, Quartiere XII Gianicolense, Roma, 00165',
 'Viale Bartolomeo Rozat, Monteverde, Quartiere XII Gianicolense, Roma, 00120',
 'Piazzale Aurelio, Monteverde, Quartiere XII Gianicolense, Roma, 00120',
 'Postamat, Via Giacomo Venezian, Rione XIII Trastevere, Municipio Roma I, Roma, 00120',
 'Via Anicia, Rione XIII Trastevere, Municipio Roma I, Roma, 00153',
 '74, Via di San

Looking good. Let's now place all this into a Pandas dataframe.

In [11]:
import pandas as pd

df_locations = pd.DataFrame({'area_code':area_id,
                             'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,area_code,Address,Latitude,Longitude,X,Y,Distance from center
0,area_1_7,"Viadotto della Magliana, Ostiense, Quartiere X...",41.841412,12.463290,289387.21939,4.635280e+06,5992.495307
1,area_1_8,"Viale del Pattinaggio, Ostiense, Quartiere X O...",41.841572,12.470510,289987.21939,4.635280e+06,5840.376700
2,area_1_9,"Garden Tre Fontane, 90, Via Laurentina, Ostien...",41.841730,12.477730,290587.21939,4.635280e+06,5747.173218
3,area_1_10,"Scuola Statale Maurizio Poggiali, Via Benedett...",41.841889,12.484950,291187.21939,4.635280e+06,5715.767665
4,area_1_11,"I.C. Poggiali Spizzichino, Via Giuseppe Berto,...",41.842047,12.492170,291787.21939,4.635280e+06,5747.173218
5,area_1_12,"451, Via di Grotta Perfetta, Roma 70, Quartier...",41.842204,12.499390,292387.21939,4.635280e+06,5840.376700
6,area_1_13,"Largo Strindberg, Roma 70, Quartiere XX Ardeat...",41.842361,12.506610,292987.21939,4.635280e+06,5992.495307
7,area_2_5,"Via Pietro Frattini, Quartiere XI Portuense, R...",41.845848,12.452275,288487.21939,4.635799e+06,5855.766389
8,area_2_6,"Piazza Fabrizio De Andrè, Quartiere XI Portuen...",41.846008,12.459495,289087.21939,4.635799e+06,5604.462508
9,area_2_7,"Via Crespina, Quartiere XI Portuense, Roma, 00146",41.846167,12.466716,289687.21939,4.635799e+06,5408.326913


...and let's now see the shape of the Dataframe.

In [12]:
df_locations.shape

(364, 7)

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on dentists in each neighborhood.

We're interested in venues in 'dentist' category. So we will include in our list only venues that have 'dentist' in category type, as we need info on  dentists in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.

In [13]:
# The code was hided.
foursquare_client_id= '*********************************************' # your Foursquare ID
foursquare_client_secret= ''*********************************************'' # your Foursquare Secret


In [14]:
# Category IDs corresponding to Italian dentists were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

dentist_office_category = '4bf58dd8d48988d178941735' # 

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Roma Capitale', '')
    address = address.replace(', Italy', '')
    return address
import requests
def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [15]:
dentist_office_category = '4bf58dd8d48988d178941735' # 

def get_dentist(lats, lons, zone):
    #dentist = {}
    #italian_dentists = {}
    dentists = []
    area_dentists = []
    print('Obtaining venues around candidate locations:', end='')
    for lat, lon, area_id in zip(lats, lons, zone):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any dentist (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, dentist_office_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        if venues==[]: 
            print(' x', end='')
            continue
        area_dentists = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            dentists_in_area=len(venues)
            dentist = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y, dentists_in_area, area_id)
            dentists.append(dentist)
        print(' .', end='')
    print(' done.')
    return dentists

dentists = get_dentist(latitudes, longitudes, area_id)
    

Obtaining venues around candidate locations: . x x . x x x . . x x . x x x x x x x . x x . x . . . x x x x x . . . . . . . x . x x x x x . . . . . x x . x x x x x x x x . x . . . x . . . x x x . x x x x x . . . x . x x . . x . x . . x x x x x . . . . . . . . x x . . . . . . x x x . . x . . . x x . . . . x . x . . x x x x . x . x . x x . . x x x x x x x x x x x x x x . x . . . . . . . . x x x . . . x x . x . x . . . x x x . . . . . . . . . . . x . x . . x x x x x x . . . . x . . . . . . . . x x x x x x . . x x . . . . . . . . . . . x x . . x x x . x . . . . . . . . . . . x . . . x . . . . . . x x . . . . . x x x . x x . . x . . x . . . . . . x x x x x x x x x x . . . x . x x x x x x x x x . x . x x . . . x x x x . x x x x x . . . x x . x x x x x x x . x . x x x x done.


In [16]:
import numpy as np

print('Total number of dentists:', len(dentists))

Total number of dentists: 425


In [17]:
dentists[100]

('5a9fb0801953f346168f0813',
 'Family Dent',
 41.87563647843966,
 12.480500936508179,
 'Via Ostiense, 4, 00154 Roma Lazio, Italia',
 127,
 290927.8343516645,
 4639037.938809394,
 2,
 'area_8_9')

Let's now see all the collected dentists in our area of interest on map.

In [18]:
map_rome = folium.Map(location=rome_center, zoom_start=13)
folium.Marker(rome_center, popup='Piazza del Campidoglio').add_to(map_rome)
for dent in dentists:
    lat = dent[2]; lon = dent[3]
    #is_italian = res[6]
    #color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rome)
map_rome

![Rome](img/2.jpg)

Looking good. So now we have all the dentists in area within few kilometers from Piazza del Campidoglio. We also know which dentists exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian dentist!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Rome that have low dentist density, particularly those with low number of Italian dentists. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every dentist within 6km from Rome center** (Piazza del Campidoglio). We have also **identified Italian dentists** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**dentist density**' across different areas of Rome - we will use **heatmaps** to identify a few promising areas close to center with low number of dentists in general (*and* no Italian dentists in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than one dentist in radius of 300 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of dentists in every area candidate**:

...let's create the DataFrame with all the dentists location features 

In [19]:
df_dentists=pd.DataFrame(dentists,columns=['venue_id','name','lat','long','location','civic_id','X','Y','area_dentists','area_code'])

In [20]:
df_dentists.head()

,venue_id,name,lat,long,location,civic_id,X,Y,area_dentists,area_code
0,5b8f418b6cd68e0031ddf0ba,Studio Dentistico Litta,41.839110,12.463570,"Viale di Val Fiorita, 86, 00144 Roma Lazio, It...",257,289402.889203,4.635024e+06,1,area_1_7
1,4ec154a5b803bca7deeb063f,Dott. Giacomuzzi,41.841188,12.481903,"Roma Lazio, Italia",264,290931.899898,4.635209e+06,2,area_1_10
2,5bae8c96e0c0c9002c1ff4b7,Eur Medical,41.843770,12.482570,"Via Francesco Acri, 32, 00142 Roma Lazio, Italia",287,290995.728527,4.635495e+06,2,area_1_10
3,5063173be4b0b8d63da72a4c,Studio Dentistico,41.848372,12.452162,"Via di Vigna Due Torri, Italia",281,288486.122184,4.636080e+06,1,area_2_5
4,58dd0a9537da1d6407b9ab69,Centro Odontoiatrico C.D.C.,41.845770,12.462350,"Via Dell' Impruneta, 164, 00146 Roma Lazio, It...",238,289323.450085,4.635766e+06,2,area_2_6


In [21]:
df_dentists_filtered=df_dentists[['area_code','area_dentists']]

In [22]:
print('We have got {} dentist offices in {} areas'.format(len(df_dentists_filtered), len(df_locations)))

We have got 425 dentist offices in 364 areas


Now, let's maerge the original hexagonal location dataset to the dentist dataset with area_code as reference

In [23]:
good_loc=pd.merge(df_dentists_filtered,df_locations,on='area_code',how='inner')

In [24]:
good_loc.head()

,area_code,area_dentists,Address,Latitude,Longitude,X,Y,Distance from center
0,area_1_7,1,"Viadotto della Magliana, Ostiense, Quartiere X...",41.841412,12.463290,289387.21939,4.635280e+06,5992.495307
1,area_1_10,2,"Scuola Statale Maurizio Poggiali, Via Benedett...",41.841889,12.484950,291187.21939,4.635280e+06,5715.767665
2,area_1_10,2,"Scuola Statale Maurizio Poggiali, Via Benedett...",41.841889,12.484950,291187.21939,4.635280e+06,5715.767665
3,area_2_5,1,"Via Pietro Frattini, Quartiere XI Portuense, R...",41.845848,12.452275,288487.21939,4.635799e+06,5855.766389
4,area_2_6,2,"Piazza Fabrizio De Andrè, Quartiere XI Portuen...",41.846008,12.459495,289087.21939,4.635799e+06,5604.462508


now we got a dataframe with the information of the hexagonal location and **how many dentist offices are inside the location** (area_dentists)

In [25]:
df_dentists.groupby('area_code').count().sort_values('area_dentists', ascending=False).head()

,venue_id,name,lat,long,location,civic_id,X,Y,area_dentists
area_code,,,,,,,,,
area_17_6,8,8,8,8,8,8,8,8,8
area_8_4,7,7,7,7,7,7,7,7,7
area_16_7,6,6,6,6,6,6,6,6,6
area_15_7,6,6,6,6,6,6,6,6,6
area_16_15,6,6,6,6,6,6,6,6,6


as you can see, area_17_6 has 8 dentist offices, while area_16_15 has 6 instead

Let's crete a map showing **heatmap / density of dentists** and try to extract some meaningfull info from that. Also, let's show **borders of Rome boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km from Piazza del Campidoglio.

In [26]:
dentist_latlons = [[dent[2], dent[3]] for dent in dentists]

In [27]:
from folium import plugins
from folium.plugins import HeatMap

map_rome = folium.Map(location=rome_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_rome) #cartodbpositron cartodbdark_matter
HeatMap(dentist_latlons).add_to(map_rome)
folium.Marker(rome_center).add_to(map_rome)
folium.Circle(rome_center, radius=1000, fill=False, color='white').add_to(map_rome)
folium.Circle(rome_center, radius=2000, fill=False, color='white').add_to(map_rome)
folium.Circle(rome_center, radius=3000, fill=False, color='white').add_to(map_rome)
#folium.GeoJson(rome_boroughs, style_function=boroughs_style, name='geojson').add_to(map_rome)
map_rome

![Rome](img/3.jpg)

Looks like a few pockets of low dentist density closest to city center can be found **south, south-west from Piazza del Campidoglio**. 

OK. Let us now **filter** those locations: we're interested only in **locations with no more than one dentist in radius of 300 meters**

Filtering only the areas where only 1 office is present means the area is popolated (not a lake, river or park) and it has the minimum number of offices, so the competition is minimal

In [28]:
good_loc=good_loc[good_loc['area_dentists']==1]

In [29]:
print('there are {} areas with only one dentist office in radius of 300 meers'.format(len(good_loc)))

there are 67 areas with only one dentist office in radius of 300 meers


In [30]:
good_loc.head()

,area_code,area_dentists,Address,Latitude,Longitude,X,Y,Distance from center
0,area_1_7,1,"Viadotto della Magliana, Ostiense, Quartiere X...",41.841412,12.463290,289387.21939,4.635280e+06,5992.495307
3,area_2_5,1,"Via Pietro Frattini, Quartiere XI Portuense, R...",41.845848,12.452275,288487.21939,4.635799e+06,5855.766389
6,area_2_9,1,"Centro Sportivo BOLA, 26, Via Luigi Perna, Qua...",41.846485,12.481157,290887.21939,4.635799e+06,5204.805472
13,area_3_11,1,"Luppolo e Farina, 65/71, Viale delle Accademie...",41.851398,12.491805,291787.21939,4.636319e+06,4714.870094
14,area_3_12,1,"Villa Ibis, 11, Piazza Lorenzo Lotto, Roma 70,...",41.851556,12.499026,292387.21939,4.636319e+06,4828.043082


Let's see how this looks on a map.

In [31]:
good_latitudes = good_loc['Latitude'].values
good_longitudes = good_loc['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_rome = folium.Map(location=rome_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_rome)
HeatMap(dentist_latlons).add_to(map_rome)
folium.Circle(rome_center, radius=6000, color='white', fill=True, fill_opacity=0.6).add_to(map_rome)
folium.Marker(rome_center).add_to(map_rome)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rome) 
#folium.GeoJson(rome_boroughs, style_function=boroughs_style, name='geojson').add_to(map_rome)
map_rome

![Rome](img/4.jpg)

Looking good. We now have a bunch of locations in Rome and we know that each of those locations has no more than one dentist in radius of 300m. Any of those locations is a potential candidate for a new office, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [32]:
map_rome = folium.Map(location=rome_center, zoom_start=13)
HeatMap(good_locations, radius=35).add_to(map_rome)
folium.Marker(rome_center).add_to(map_rome)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rome)
#folium.GeoJson(rome_boroughs, style_function=boroughs_style, name='geojson').add_to(map_rome)
map_rome

![Rome](img/5.jpg)

Looking good. What we have now is a clear indication of zones with low number of dentists in vicinity, and *no* Italian dentists at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [33]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = good_loc[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]


let's include the cluster information to the DataFrame

In [34]:
good_loc['clusters']= kmeans.labels_

In [35]:
good_loc.head(10)

,area_code,area_dentists,Address,Latitude,Longitude,X,Y,Distance from center,clusters
0,area_1_7,1,"Viadotto della Magliana, Ostiense, Quartiere X...",41.841412,12.463290,289387.21939,4.635280e+06,5992.495307,11
3,area_2_5,1,"Via Pietro Frattini, Quartiere XI Portuense, R...",41.845848,12.452275,288487.21939,4.635799e+06,5855.766389,11
6,area_2_9,1,"Centro Sportivo BOLA, 26, Via Luigi Perna, Qua...",41.846485,12.481157,290887.21939,4.635799e+06,5204.805472,11
13,area_3_11,1,"Luppolo e Farina, 65/71, Viale delle Accademie...",41.851398,12.491805,291787.21939,4.636319e+06,4714.870094,1
14,area_3_12,1,"Villa Ibis, 11, Piazza Lorenzo Lotto, Roma 70,...",41.851556,12.499026,292387.21939,4.636319e+06,4828.043082,1
15,area_3_13,1,"Via Filippo Zaniberti, Roma 70, Quartiere XX A...",41.851713,12.506247,292987.21939,4.636319e+06,5010.987927,1
21,area_4_7,1,"Roma Tre - Facoltà di Scienze Matematiche, Fis...",41.855519,12.466346,289687.21939,4.636839e+06,4419.275959,11
28,area_4_11,1,"Via dell'Arcadia, Quartiere XX Ardeatino, Roma...",41.856153,12.495233,292087.21939,4.636839e+06,4253.234064,1
29,area_4_13,1,"Mausoleo delle Fosse Ardeatine, Largo Martiri ...",41.856467,12.509676,293287.21939,4.636839e+06,4657.252409,1
30,area_5_5,1,"Piazza Alfonso Neuschuler, Quartiere XII Giani...",41.859794,12.448107,288187.21939,4.637358e+06,4714.870094,4


In [36]:
map_rome = folium.Map(location=rome_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_rome)
HeatMap(dentist_latlons).add_to(map_rome)
#folium.Circle(rome_center, radius=6000, color='white', fill=True, fill_opacity=0.4).add_to(map_rome)
folium.Marker(rome_center).add_to(map_rome)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=1000, color='green', fill=True, fill_opacity=0.25).add_to(map_rome) 
#for lat, lon in zip(good_latitudes, good_longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rome)
#folium.GeoJson(rome_boroughs, style_function=boroughs_style, name='geojson').add_to(map_rome)
map_rome

![Rome](img/6.jpg)

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

This concludes our analysis. We have created 15 areas representing centers of zones containing locations with low number of dentists nearby, all zones being fairly close to city center (all less than 6km from center). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential dentist locations.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of dentists in Rome (425 in our initial area of interest which was 12x12km around Piazza del Campidoglio), there are pockets of low dentist density, 67 areas with only one dentist office in a radius of 300 meters.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates.

Result of all this is 15 zones containing largest number of potential new dentist locations to start a deeper analysis.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Rome areas close to center with low number of dentists in order to aid stakeholders in narrowing down the search for optimal location for a new dentist office. By calculating dentist density distribution from Foursquare data we have generated extensive collection of locations which satisfy some basic requirements regarding existing nearby dentists with low density. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal dentist location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like density of population.